In [ ]:
!nvidia-smi

In [ ]:
!pip install -U openmim
!mim install mmcv-full==1.6.0
!pip install wandb
!git clone https://github.com/main-2983/Polyp.git
%cd Polyp
%pip install -v -e .

!gdown --id 1o8OfBvYE6K-EpDyvzsmMPndnUMwb540R
!gdown --id 1lODorfB33jbd-im-qrtUgWnZXxB94F55
!gdown --id 1te9-crPNrl-M6Hs3cUxZaLBNmGIXpgGo
!gdown --id 1Yo8iKmSWNYpFNj65K6u7XndO3QRqNNup
!gdown --id 1aJDS8yVKfecqLS3xvEJikjL_whXEnICM
!unzip TrainDataset.zip
!unzip TestDataset.zip

!mkdir checkpoints
!mkdir pretrained
!mv mit_b1_mmseg.pth pretrained/
!mv mit_b3_mmseg.pth pretrained/
!mv mit_b4_mmseg.pth pretrained/

In [ ]:
!pip3 uninstall opencv-python-headless==4.5.5.62 -y
!pip3 install opencv-python-headless==4.5.2.52
!pip3 install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip3 install -U git+https://github.com/qubvel/segmentation_models.pytorch
!pip3 install pytorch_lightning

In [ ]:
!git checkout Quan

Branch 'quan' set up to track remote branch 'quan' from 'origin'.
Switched to a new branch 'quan'


In [ ]:
!python main.py

[Errno 2] No such file or directory: 'Polyp'
/content/Polyp
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1615: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/dropout/cutout.py:52: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,
Experiment will be saved to /content/Polyp/runs/test
Global seed set to 2022
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: quan-nv (ssl-online). Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./wandb/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path ./wandb/wandb/ wasn't writable, using system temp directory
wandb: Tracking run with wandb version 0.13.2
wandb: Run data is saved locally in /tmp/wand

In [ ]:
# model segformerb4 settings

from mmseg.models import build_segmentor
from mmcv.runner.optimizer import build_optimizer

def segformer():
    model = dict(
        type='SunSegmentor',
        backbone=dict(
            type='MixVisionTransformer',
            in_channels=3,
            embed_dims=64,
            num_stages=4,
            num_layers=[2, 2, 2, 2],
            num_heads=[1, 2, 5, 8],
            patch_sizes=[7, 3, 3, 3],
            sr_ratios=[8, 4, 2, 1],
            out_indices=(0, 1, 2, 3),
            mlp_ratio=4,
            qkv_bias=True,
            drop_rate=0.0,
            attn_drop_rate=0.0,
            drop_path_rate=0.1,
            pretrained="/content/Polyp/pretrained/mit_b1_mmseg.pth"),
        decode_head=dict(
            type='UPerHeadV3',
            in_channels=[64, 128, 320, 512],
            in_index=[0, 1, 2, 3],
            channels=128,
            dropout_ratio=0.1,
            num_classes=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            align_corners=False,
            loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
        # model training and testing settings
        train_cfg=dict(),
        test_cfg=dict(mode='whole'))
    model = build_segmentor(model)
    model.init_weights()
    return model
    

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6
model = segformer()
import torch
len(model(torch.rand((2, 3, 512, 512))))

/content/Polyp/mmseg/models/backbones/mit.py:365: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/content/Polyp/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/hsigmoid.py:37: UserWarning: In MMCV v1.4.4, we modified the default value of args to align with PyTorch official. Previous Implementation: Hsigmoid(x) = min(max((x + 1) / 2, 0), 1). Current Implementation: Hsigmoid(x) = min(max((x + 3) / 6, 0), 1).
  'In MMCV v1.4.4, we modified the default value of args to align '
2022-08-29 07:31:39,570 - mmcv - INFO - initialize MixVisionTransformer with 

5

In [ ]:
count_parameters(model)

18.139873